In [2]:
import os
import json
import cv2
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [3]:
# Функція для парсингу JSON і завантаження зображень поштучно
def parse_data(json_path, images_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    for item in data['images']:
        img_path = os.path.join(images_path, item['file_name'])
        image = cv2.imread(img_path)
        image = cv2.resize(image, (224, 224))

        # Створюємо one-hot вектор для категорій
        label = np.zeros(len(data['categories']))
        for ann in data['annotations']:
            if ann['image_id'] == item['id']:
                label[ann['category_id']] = 1

        yield image, label


In [4]:
# Функція для створення tf.data.Dataset
def create_dataset(json_path, images_path, batch_size):
    dataset = tf.data.Dataset.from_generator(
        lambda: parse_data(json_path, images_path),
        output_signature=(
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.uint8),
            tf.TensorSpec(shape=(3,), dtype=tf.float32)
        )
    )
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))  # Нормалізація зображень
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)  # Покращення продуктивності
    return dataset

In [5]:
# Створення наборів даних для тренування, валідації та тестування
batch_size = 32

train_dataset = create_dataset('D:/My_repo/Data_Science/project/train/_annotations.coco.json', 
                               'D:/My_repo/Data_Science/project/train', 
                               batch_size)

valid_dataset = create_dataset('D:/My_repo/Data_Science/project/valid/_annotations.coco.json', 
                               'D:/My_repo/Data_Science/project/valid', 
                               batch_size)

test_dataset = create_dataset('D:/My_repo/Data_Science/project/test/_annotations.coco.json', 
                              'D:/My_repo/Data_Science/project/test', 
                              batch_size)

In [6]:
# Кількість класів
num_classes = 3

# Створення базової моделі
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Додавання глобального шару агрегації та повнозв'язного шару
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)



In [7]:
# Створення моделі
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [8]:
# Заморожування базових шарів
for layer in base_model.layers:
    layer.trainable = False

In [9]:
# Компіляція моделі
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Визначення кількості кроків на епоху
train_steps_per_epoch = len(os.listdir('D:/My_repo/Data_Science/project/train')) // batch_size
valid_steps_per_epoch = len(os.listdir('D:/My_repo/Data_Science/project/valid')) // batch_size

In [11]:
# Тренування моделі
model.fit(train_dataset, validation_data=valid_dataset, epochs=10, steps_per_epoch=train_steps_per_epoch, validation_steps=valid_steps_per_epoch)

Epoch 1/10
 53/271 ━━━━━━━━━━━━━━━━━━━━ 13:26 4s/step - accuracy: 0.6294 - loss: 1.2393

In [ ]:
# Оцінка точності на тестовому наборі
loss, accuracy = model.evaluate(test_dataset)
print(f'Втрата на тестовому наборі: {loss:.4f}')
print(f'Точність на тестовому наборі: {accuracy:.4f}')